In [2]:

import numpy as np
import cPickle as pickle
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [3]:
import gym
env = gym.make('CartPole-v0')

[2017-02-06 22:55:35,412] Making new env: CartPole-v0


In [4]:
env.reset()
H=10
batch_size = 5
learning_rate = 1e-2
gamma = 0.99 # more focus on future reward

D = 4

In [9]:
tf.reset_default_graph()
observations = tf.placeholder(tf.float32,shape= [None,D], name = "input_x")
W1 = tf.get_variable('W1',shape = [D,H], initializer = tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable('W2',shape = [H,1], initializer = tf.contrib.layers.xavier_initializer())
probability = tf.nn.sigmoid(tf.matmul(layer1,W2))

tvars = tf.trainable_variables() ## store variabes W1 W2, list, list[0] W1 shape:[4,10] 
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal") ## ??

loglik = tf.log(input_y*(input_y-probability)+(1-input_y)*(input_y+probability))
loss = -tf.reduce_mean(loglik*advantages)
newGrads = tf.gradients(loss,tvars)

adam = tf.train.AdamOptimizer(learning_rate=learning_rate)
W1Grad = tf.placeholder(tf.float32,name = 'batch_grad1')
W2Grad = tf.placeholder(tf.float32,name = 'batch_grad2')
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

In [6]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0,len(r))):
        running_add = running_add*gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [10]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000


In [11]:

init = tf.global_variables_initializer()
with tf.Session() as sess:
    rendering  = False
    sess.run(init)
    observation = env.reset()
    
    gradBuffer = sess.run(tvars)
    
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad*0
    
    while episode_number<=1000:
        
        if reward_sum/batch_size > 100 or rendering == True : 
            env.render()
            rendering = True
        
        x = np.reshape(observation,[1,D])
        tfprob = sess.run(probability,feed_dict={observations:x})
        
        action = 1 if np.random.uniform()<tfprob else 0
        xs.append(x)
        y=1 if action == 0 else 0
        ys.append(y)
        
        observation,reward,done,_ = env.step(action)
        reward_sum += reward
        drs.append(reward)
        
        if done:
            episode_number += 1
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            xs,hs,dlogps,drs,ys = [],[],[],[],[]
            
            discounted_epr = discount_rewards(drs)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
            
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                
                print 'Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size)
                
                if reward_sum/batch_size > 200: 
                    print "Task solved in",episode_number,'episodes!'
                    break
                    
                reward_sum = 0
            
            observation = env.reset()

Average reward for episode 21.600000.  Total average reward 21.600000.
Average reward for episode 31.000000.  Total average reward 21.694000.
Average reward for episode 31.000000.  Total average reward 21.787060.
Average reward for episode 27.200000.  Total average reward 21.841189.
Average reward for episode 31.600000.  Total average reward 21.938778.
Average reward for episode 29.200000.  Total average reward 22.011390.
Average reward for episode 30.400000.  Total average reward 22.095276.
Average reward for episode 19.200000.  Total average reward 22.066323.
Average reward for episode 20.600000.  Total average reward 22.051660.
Average reward for episode 22.200000.  Total average reward 22.053143.
Average reward for episode 33.800000.  Total average reward 22.170612.
Average reward for episode 22.200000.  Total average reward 22.170906.
Average reward for episode 34.000000.  Total average reward 22.289197.
Average reward for episode 49.000000.  Total average reward 22.556305.
Averag

In [12]:

env.render()

In [84]:
a.append(0.05)

In [92]:
np.vstack(a).shape

(5, 1)

In [156]:
env.render()